<a href="https://www.kaggle.com/code/nigamshitij/goodreads-book-rating-data-analysis?scriptVersionId=192980193" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
pip install beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
# setup

import requests
from bs4 import BeautifulSoup

url = "https://www.goodreads.com/shelf/show/fantasy"
container = ["title", "book_url", "author", "avg_rating", "ratings", "published_year"]

In [8]:
# import requests
 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
 
response = requests.get(url, headers=headers)

In [9]:
# response = requests.get(url)
# print(response)
html_content = response.text
# print(html_content)
soup = BeautifulSoup(html_content, "html.parser")
# print(soup.prettify())

In [10]:
total_items_info = soup.find("div", class_="mediumText")
print(total_items_info)

<div class="mediumText">
<span class="smallText">
Showing 1-50 of 100,000
</span>
</div>


In [11]:
items = total_items_info.get_text().strip()
total_items = int(items.split()[-1].replace(",",""))
print(total_items)

100000


In [36]:
# temp fields
title = []
url_list = []
authors = []
avg_ratings = []
rating = []
year = []

# define genre
genre = "aeroplanes"

# define headers, url, other hyperparameters for request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
base_url = "https://www.goodreads.com/shelf/show/"+genre
items_per_page = 50
total_pages = (total_items + items_per_page - 1)
max_pages_to_scrape = 25

# iterate through pages
for page in range(1, min(max_pages_to_scrape, total_pages) + 1):
    # Construct the URL for the current page
    url = f"{base_url}?page={page}"
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, "html.parser")
    book_elements = soup.find_all("div", "elementList")
    
    # Iterate through each book element on the current page
    for book_element in book_elements:
        # Use try-except blocks to handle potential errors if elements are missing
        try:
            # Extract book details
            book_title = book_element.find("a", "bookTitle").text
            book_url = "https://www.goodreads.com" + book_element.find("a", "bookTitle").get("href")
            author = book_element.find("a", "authorName").text
            rating_text = book_element.find("span", "greyText smallText").text.split()
            avg_rating = rating_text[2]
            ratings = rating_text[4]
            published_year = rating_text[-1] if len(rating_text) == 9 else ""
    
            # Append the extracted data to their respective lists
            title.append(book_title)
            url_list.append(book_url)
            authors.append(author)
            avg_ratings.append(avg_rating)
            rating.append(ratings)
            year.append(published_year)
        except AttributeError:
            # Handle the case where an element is not found
            print(f"Skipping a book on page {page} due to missing data.")

# store data from temp fields into dataframe          
good_reads = pd.DataFrame({
    "Title": title,
    "URL": url_list,
    "Authors": authors,
    "Avg Ratings": avg_ratings,
    "Rating": rating,
    "Published_year": year
})

# print head to check data
print(good_reads.head())

# store file into genre CSV
filename = "goodreads_" + genre + ".csv"
good_reads.to_csv(filename, index=False)

# print average rating
avg_rating = (good_reads["Avg Ratings"].astype(float).sum())/len(good_reads.index)
print(f"Average rating of {genre}: {avg_rating}")

Skipping a book on page 1 due to missing data.
Skipping a book on page 1 due to missing data.
Skipping a book on page 2 due to missing data.
Skipping a book on page 2 due to missing data.
Skipping a book on page 3 due to missing data.
Skipping a book on page 3 due to missing data.
Skipping a book on page 4 due to missing data.
Skipping a book on page 4 due to missing data.
Skipping a book on page 5 due to missing data.
Skipping a book on page 5 due to missing data.
Skipping a book on page 6 due to missing data.
Skipping a book on page 6 due to missing data.
Skipping a book on page 7 due to missing data.
Skipping a book on page 7 due to missing data.
Skipping a book on page 8 due to missing data.
Skipping a book on page 8 due to missing data.
Skipping a book on page 9 due to missing data.
Skipping a book on page 9 due to missing data.
Skipping a book on page 10 due to missing data.
Skipping a book on page 10 due to missing data.
Skipping a book on page 11 due to missing data.
Skipping a

In [32]:
good_reads.head()

,Title,URL,Authors,Avg Ratings,Rating,Published_year
0,Harry Potter and the Sorcerer’s Stone (Harry P...,https://www.goodreads.com/book/show/42844155-h...,J.K. Rowling,4.47,"10,309,694",1997
1,Harry Potter and the Chamber of Secrets (Harry...,https://www.goodreads.com/book/show/15881.Harr...,J.K. Rowling,4.43,"4,034,077",1998
2,Harry Potter and the Prisoner of Azkaban (Harr...,https://www.goodreads.com/book/show/5.Harry_Po...,J.K. Rowling,4.58,"4,310,547",1999
3,"The Hobbit (The Lord of the Rings, #0)",https://www.goodreads.com/book/show/5907.The_H...,J.R.R. Tolkien,4.29,"4,083,426",1937
4,Harry Potter and the Goblet of Fire (Harry Pot...,https://www.goodreads.com/book/show/6.Harry_Po...,J.K. Rowling,4.57,"3,791,953",2000


In [21]:
top_genre_url_1 = "https://www.goodreads.com/genres/list?filter=top-level"
top_genre_url_2 = "https://www.goodreads.com/genres/list?filter=top-level"

genres = []
genre_urls = []
# genre_books = []

for page in range(1, min(2, total_pages) + 1):
    # Construct the URL for the current page
    url = f"{top_genre_url_1}&page={page}"
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, "html.parser")
    book_genres = soup.find_all("div", "shelfStat")
    
    # Iterate through each book element on the current page
    for book_genre in book_genres:
        # Use try-except blocks to handle potential errors if elements are missing
        try:
            # Extract book details
            genre = book_genre.find("a", "mediumText").text
            genre_url = "https://www.goodreads.com" + book_genre.find("a", "mediumText").get("href")
#             genre_books = book_genre.find("a", "mediumText").text
    
            # Append the extracted data to their respective lists
            genres.append(genre)
            genre_urls.append(genre_url)
#             genre_books.append(genre_books)
        except AttributeError:
            # Handle the case where an element is not found
            print(f"Skipping a genre on page {page} due to missing data.")

In [22]:
genres = pd.DataFrame({
    "Genre": genres,
    "URL": genre_urls,
#     "Total Books": genre_books
})

In [23]:
genres.head()

,Genre,URL
0,40k,https://www.goodreads.com/genres/40k
1,adolescence,https://www.goodreads.com/genres/adolescence
2,adult,https://www.goodreads.com/genres/adult
3,aeroplanes,https://www.goodreads.com/genres/aeroplanes
4,amish,https://www.goodreads.com/genres/amish


In [24]:
genres.to_csv("goodreads_genres.csv", index=False)

In [51]:
url_test = "https://www.goodreads.com/shelf/show/"+genres['Genre'][0]
print(url_test)

https://www.goodreads.com/shelf/show/40k


In [59]:
main_df = pd.DataFrame(columns = ["Title", "URL", "Authors", "Avg Ratings", "Rating", "Published_year", "Genre"])

def scrape_per_genre(genre):
    # function temp variables
    title = []
    url_list = []
    authors = []
    avg_ratings = []
    rating = []
    year = []
    genre = []

    for page in range(1, min(5, total_pages) + 1):
    # Construct the URL for the current page
    
        url = "https://www.goodreads.com/shelf/show"
        genre_url = f"{base_url}/{genre}?page={page}"
        response = requests.get(genre_url, headers=headers).text
        soup = BeautifulSoup(response, "html.parser")
        book_elements = soup.find_all("div", "elementList")

        # Iterate through each book element on the current page
        for book_element in book_elements:
            # Use try-except blocks to handle potential errors if elements are missing
            try:
                # Extract book details
                book_title = book_element.find("a", "bookTitle").text
                book_url = "https://www.goodreads.com" + book_element.find("a", "bookTitle").get("href")
                author = book_element.find("a", "authorName").text
                rating_text = book_element.find("span", "greyText smallText").text.split()
                avg_rating = rating_text[2]
                ratings = rating_text[4]
                published_year = rating_text[-1] if len(rating_text) == 9 else ""

                # Append the extracted data to their respective lists
                title.append(book_title)
                url_list.append(book_url)
                authors.append(author)
                avg_ratings.append(avg_rating)
                rating.append(ratings)
                year.append(published_year)
                genre.append(genre)
            except AttributeError:
                # Handle the case where an element is not found
                print(f"Skipping a book on page {page} due to missing data.")

    temp_df = pd.DataFrame({
        "Title": title,
        "URL": url_list,
        "Authors": authors,
        "Avg Ratings": avg_ratings,
        "Rating": rating,
        "Published_year": year,
        "Genre": genre
    })
    
    return temp_df

genre_df = scrape_per_genre(genres['Genre'][0])

genre_df.head()

Skipping a book on page 1 due to missing data.
Skipping a book on page 1 due to missing data.
Skipping a book on page 2 due to missing data.
Skipping a book on page 2 due to missing data.
Skipping a book on page 3 due to missing data.
Skipping a book on page 3 due to missing data.
Skipping a book on page 4 due to missing data.
Skipping a book on page 4 due to missing data.
Skipping a book on page 5 due to missing data.
Skipping a book on page 5 due to missing data.



KeyboardInterrupt



In [ ]:
genre_df.head()